In [1]:
from simple.binance import getHistMonth
from simple.plotly import chartFigure, updateLines
import pandas as pd
from tqdm.auto import tqdm
from multiprocessing import Pool

from statsmodels.tsa.stattools import adfuller
from ipyslickgrid import show_grid
from ipywidgets import VBox

In [2]:
tickers = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'BCHUSDT', 'TRXUSDT', 'DOGEUSDT', 'ETCUSDT', 'XRPUSDT']
frame = '1h'

In [3]:
def getClose(ticker):
    return getHistMonth('2022-03', '2022-03', ticker, frame, close_only=True)

In [4]:
F = pd.concat(tqdm(Pool(64).imap(getClose, tickers), total=len(tickers)), axis=1)
F

  0%|          | 0/8 [00:00<?, ?it/s]

,BTCUSDT,ETHUSDT,LTCUSDT,BCHUSDT,TRXUSDT,DOGEUSDT,ETCUSDT,XRPUSDT
DT,,,,,,,,
2022-03-01 00:00:00,43548.7,2947.44,114.61,339.28,0.06244,0.13379,30.992,0.7848
2022-03-01 01:00:00,43254.1,2917.25,112.93,333.71,0.06188,0.13260,30.205,0.7711
2022-03-01 02:00:00,43148.9,2917.35,112.76,331.70,0.06180,0.13224,30.257,0.7687
2022-03-01 03:00:00,43140.8,2911.82,112.52,331.01,0.06128,0.13247,30.034,0.7745
2022-03-01 04:00:00,43323.5,2924.33,112.99,331.54,0.06173,0.13285,30.270,0.7765
...,...,...,...,...,...,...,...,...
2022-03-31 19:00:00,45578.1,3282.42,124.22,376.87,0.07588,0.13808,47.161,0.8121
2022-03-31 20:00:00,45741.9,3298.07,124.85,385.40,0.07534,0.13951,47.489,0.8161
2022-03-31 21:00:00,45736.1,3286.20,124.36,382.35,0.07478,0.13913,47.358,0.8173


In [5]:
D = F.apply(adfuller).T
D.columns=['ADF', 'p-value', 'x', 'y', 'z', 'p']
D

,ADF,p-value,x,y,z,p
BTCUSDT,-1.015199,0.747681,0,743,"{'1%': -3.439181811684251, '5%': -2.8654378076...",10103.355324
ETHUSDT,-0.430694,0.904846,0,743,"{'1%': -3.439181811684251, '5%': -2.8654378076...",6341.581978
LTCUSDT,-0.855991,0.80212,0,743,"{'1%': -3.439181811684251, '5%': -2.8654378076...",1732.648916
BCHUSDT,-0.069646,0.952379,4,739,"{'1%': -3.439229783394421, '5%': -2.8654589481...",3406.045515
TRXUSDT,0.617199,0.988032,5,738,"{'1%': -3.4392418578608983, '5%': -2.865464269...",-9404.304884
DOGEUSDT,-0.955809,0.769039,1,742,"{'1%': -3.4391937559530965, '5%': -2.865443071...",-7984.455729
ETCUSDT,0.163874,0.970161,10,733,"{'1%': -3.439302726125018, '5%': -2.8654910924...",1026.731423
XRPUSDT,-1.670603,0.446319,0,743,"{'1%': -3.439181811684251, '5%': -2.8654378076...",-5419.942462


In [6]:
fig = chartFigure(Close=dict(color='lightblue', width=3))

In [7]:
def on_changed(event, grid):
    """Redraw chart with selected ticker's data"""
    
    changed = grid.get_changed_df()
    k = event['new'][0]
    selected = changed.iloc[k:k + 1].to_dict('records')[0]
    ticker = selected['index']
    updateLines(fig, Close=F[ticker])

grid = show_grid(D.reset_index(), grid_options={'editable': False, 'forceFitColumns': True, 'multiSelect': False},
                 column_options={'defaultSortAsc': False})
grid.on('selection_changed', on_changed)

In [8]:
VBox([fig, grid])

    'data': [{'hovertext': '',
              'line': {'color': 'lightbl…